In [ ]:
import sqlite3, json, math, datetime as dt, pandas as pd
import matplotlib.dates as mdates

# ---------------------------------------------------------------------------
# 1.  Load and flatten reviews
# ---------------------------------------------------------------------------
DB_PATH = "mochi.db"  # adjust if needed
MS_DAY = 86_400_000


def ms_to_dt(ms_str):  # "~t1723014000000" -> datetime
    return dt.datetime.utcfromtimestamp(int(ms_str[2:]) / 1000)


def interval_of(r):
    if "~:interval" in r:
        return r["~:interval"]
    if "~:due" in r and "~:date" in r:
        return max(
            round((int(r["~:due"][2:]) - int(r["~:date"][2:])) / MS_DAY), 0
        )
    return math.nan


rows, deck_name = [], {}

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()

    # deck lookup
    cur.execute(
        'SELECT json FROM "by-sequence" WHERE json LIKE \'%"type":"deck"%\''
    )
    for (doc,) in cur.fetchall():
        td = json.loads(doc)["transit-data"]
        deck_name[td["~:id"]] = td["~:name"]

    # cards + reviews
    cur.execute(
        'SELECT json FROM "by-sequence" WHERE json LIKE \'%"type":"card"%\''
    )
    for (doc,) in cur.fetchall():
        td = json.loads(doc)["transit-data"]
        deck = td["~:deck-id"]
        for r in td["~:reviews"]:
            rows.append(
                {
                    "deck": deck_name.get(deck, deck),
                    "card_id": td["~:id"],
                    "when": ms_to_dt(r["~:date"]),
                    "interval": interval_of(r),
                }
            )

reviews = pd.DataFrame(rows)
reviews["when"] = pd.to_datetime(reviews["when"])  # ensure pandas dtype

# ---------------------------------------------------------------------------
# 2.  Forecast workload & per-day head-room
# ---------------------------------------------------------------------------
CAP_REVIEW = 30  # max reviews you’re willing to do on ANY day
HORIZON_DAYS = 30

# latest review per card  → current interval → next due date
latest = (
    reviews.sort_values("when")
    .groupby("card_id", as_index=False)
    .tail(1)
    .dropna(subset=["interval"])
)
latest["due"] = latest["when"] + pd.to_timedelta(latest.interval, unit="D")

# Series indexed by date
rng = pd.date_range(dt.date.today(), periods=HORIZON_DAYS, name="date")
due_cnt = (
    latest.groupby(latest["due"].dt.date)
    .size()
    .reindex(rng.date, fill_value=0)
)

headroom_per_day = (CAP_REVIEW - due_cnt).clip(lower=0)  # never negative

# ---------------------------------------------------------------------------
# 3.  Display results
# ---------------------------------------------------------------------------
dashboard = pd.DataFrame(
    {"scheduled_reviews": due_cnt, "headroom": headroom_per_day}
)
print("Cards scheduled & daily head-room (next 30 days)")
display(dashboard)

# Recommended new-card plan = exactly that head-room Series
print("\nSuggested additions")
for date, space in headroom_per_day.items():
    print(f"{date}: add ≤ {space:3d} new card(s)")

# Optional plot
# try:
#     import matplotlib.pyplot as plt

#     fig, ax = plt.subplots(figsize=(10, 4))
#     due_cnt.plot(kind="line", label="Scheduled reviews", ax=ax)
#     headroom_per_day.plot(kind="line", label="Head-room", ax=ax)
#     ax.set_title("30-day workload vs. new-card capacity")
#     ax.set_ylabel("Cards")
#     ax.legend()
#     plt.tight_layout()
# except ImportError:
#     pass

fig, ax = plt.subplots(figsize=(12, 4))
due_cnt.plot(kind="line", label="Scheduled reviews", ax=ax)
headroom_per_day.plot(kind="line", label="Head-room", ax=ax)

# NEW: show *every* day as an x-tick
ax.xaxis.set_major_locator(mdates.DayLocator())  # one tick per day
ax.xaxis.set_major_formatter(
    mdates.DateFormatter("%Y-%m-%d")
)  # yyyy-mm-dd labels
ax.tick_params(axis="x", rotation=90)  # slanted labels

ax.set_title("30-day workload vs. new-card capacity")
ax.set_ylabel("Cards")
ax.legend()
plt.tight_layout()

TypeError: 'int' object is not subscriptable